# Tarea 4:  La ruta al caos de doblamiento de periodo.

## Rodoflo Arturo González Trillo

**Ejercicio 1:**

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)$, donde la órbita de periodo $2^n$ nace. Es decir, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.

*Hint:* Para realizar este ejercicio deben calcular el atractor para varias valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Sin embargo, van a requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 

Una opción, que tiene ciertos inconvenientes numéricos que también ciertas ventajas se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ usando los polinomios $Q_c^p(x)$ y diferenciación automática.

$f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}}$

Las siguientes funciones nos permiten desarrollar nuestro algoritmo:

In [1]:
#Función desarrollada por el profesor para encontrar los ciclos estables:

function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0    #No verifica si it está dentro del vector.
    end
    
    return nothing
end

ciclosestables! (generic function with 1 method)

In [2]:
#Función del profesor para graficar.

"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""

function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        ff[:,ic] = xx
        #Agregar una instrucción para que diga cuantos valores de bifurcación hay.
    end
    
    return ff
end

diagbifurc (generic function with 1 method)

In [3]:

"""
    eliminar_valrep_ordenar!(vector)

Esta función toma una variable de tipo ´Array´, la ordena y
quita los elementos repetidos, considerando un margen de 
error.

"""

function eliminar_valrep_ordenar!(vector)
    
    N=length(vector)
    
    sort!(vector)  #función para ordenar.
    
    k = 1
    ϵ = 1e-04 #margen de error
    
    for i in 2:N
        if abs(vector[k]-vector[k+1]) <= ϵ  #Obseva si los valores difieren más allá de epsilón.
            deleteat!(vector,k)             #Elimina el valor repetido.
        else
            k=k+1
        end     
    end
    
    vector
    
end

eliminar_valrep_ordenar! (generic function with 1 method)

In [4]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-2.0

ff = diagbifurc(Qc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';

Queremos el valor inicial $f_0$ y $c_0$:

In [5]:
nout = 256
nit = 1000
f0 = Vector{Float64}(256)
c0=0.25

ciclosestables!(f0, Qc, nit, nout, c0)
eliminar_valrep_ordenar!(f0)

f0=f0[1]

0.4992094301416824

Ahora encontramos el valor de la bifurcación con:

In [6]:

"""
    primera_bifurcacion(f, nit, nout, crange)

Dando una función de la forma `f(x,c)`, después de
itera el mapeo `nit+nout`, evaluando sólo las úl-
timo valores `nout`, arrojando los dos primeros
puntos donde la función se bifurca en un rango 
`crange.`

"""


function primera_bifurcacion(f, nit, nout, crange)
    
    #Variables para almacenar el resultado.
    rc=Float64[]
    rf=Float64[]
    
    #Vectores vacíos para la función ciclosestables!
    xx1 = Vector{Float64}(nout)
    xx2 = Vector{Float64}(nout)
    
    #vector inicial    
    ciclosestables!(xx1, f, nit, nout, first(crange))
    eliminar_valrep_ordenar!(xx1)
    
    #Esta variable sirve como condición para detener la función.
    j=0
    
    #Se define con un ciclo for para usar todo el rango.
    for ic in 2:length(crange)
        c = crange[ic]
        
        #Uso de las funciones antes definidas.
        #Define otro vector para comparar.
        ciclosestables!(xx2, f, nit, nout, c)
        eliminar_valrep_ordenar!(xx2)
        
        #lo único que hace es comparar la diferencia en longitud de
        #los vectores.
        #Si uno es más largo que el otro, registra en que _c_ pasa y 
        #el promedio del valor de f en ambos vectores.
        if length(xx1)!= length(xx2)
            push!(rc, c)
            push!(rf,(first(xx1)+first(xx2))/2)
            
            #Esta zona es para detener la función después de obtener los
            #primeros 2 puntos de bifurcación el el intervalo.
            
            j=j+1
            if j==2
                break
            end
            
            
        end
        
        #Intercambio de valores, y regreso de xx2 a un estado en que la función
        #ciclos_estables! pueda trabajar.
        xx1 = xx2
        xx2 = Vector{Float64}(nout)
    
    end
    #Resultados.
    return  rc,rf
end


primera_bifurcacion (generic function with 1 method)

Estos valores son las primeras dos bifuracaciones encontradas:

In [7]:
c12,f12=primera_bifurcacion(Qc, 1000, 5, crange)

([-0.7431640625,-1.2470703125],[-0.49635449586843317,-1.2046592154446776])

Graficamos:

In [8]:
figure(figsize=(6,4))
plot(cc, ff, "b,")

plot(c0,f0,"go",label=L"f_0")

plot(c12,f12,"ro",label=L"f_1 y f_2")

plot([c0,c12[2],c12[2],c0,c0],[f0,f0,f12[2],f12[2],f0],"y-")



plot()
xlabel(L"c")
ylabel(L"x_\infty")
legend()
title("Fig. 2")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[8], in expression starting on line 1

El recuadro amarillo lo hemos trazado con los puntos $0$ y $2$, servirá como referencia para acotar los límites de la función, bajo la suposición de que se trata de una gráfica autosimilar y hacer el algoritmo más eficiente. Sólo seguirmos los puntos más inferiores de Figura 2.

Se observa en la línea roja horizontal, que arriba de $f_max$ queda una rama y abajo la otra. Lo mismo pasa a la izquierda y a la derecha de $c_max$. Ésto es útil porque se puede ir descartando la parte del gráfico que corresponde a la otra rama en cada valor $c$ de bifurcación encontrado.

Se pretende hacer un algoritmo recursivo que encuentre el valor de cada bifurcación, acotando como se muestra en figura 2, primero considere primero el área dentro de las líneas verdes para calcular la primera raíz, luego la de las líneas rojas, etc.

Para tal próposito hacemos un nuevo método de `eliminar_valrep_ordenar!`, que descarte tambien valores fuera de los límites de $f$ deseados:

In [9]:

"""
    eliminar_valrep_ordenar!(vector,limit,ϵ)

Esta función toma una variable de tipo ´Array´, la ordena
quita los elementos fuera del intervalo `limit`, y quita los
elementos repetidos considerando un margen de error `ϵ`.

"""

function eliminar_valrep_ordenar!(vector,limit,ϵ)
    
    N=length(vector)
    
    sort!(vector) #Ordena el vector.
    sort!(limit)
    
    k = 1
    
    for i in 2:N
        if vector[k]<limit[1] || vector[k]>limit[2]  #Observa si el valor está dentro de los límites.
            deleteat!(vector,k)            
        elseif abs(vector[k]-vector[k+1]) <= ϵ  #Obseva si los valores difieren más allá de epsilón.
            deleteat!(vector,k)             #Elimina el valor repetido.            
        else
            k=k+1
        end     
    end
    
    vector
    
end

eliminar_valrep_ordenar! (generic function with 2 methods)

Y otro método también para la función `primera_bifurcación`

In [10]:

"""
    primera_bifurcacion(f, nit, nout, crange)

Dando una función de la forma `f(x,c)`, después de
itera el mapeo `nit+nout`, evaluando sólo las úl-
timo valores `nout`, arrojando los dos primeros
puntos donde la función se bifurca en un rango 
`crange`, para valores dentro de `flimit` con una
tolerancia `ϵ`.

"""

function primera_bifurcacion(f, nit, nout, crange, flimit, ϵ)
        
    rc=Float64[]
    rf=Float64[]
    
    xx1 = Vector{Float64}(nout)
    xx2 = Vector{Float64}(nout)
    
    #vector inicial    
    ciclosestables!(xx1, f, nit, nout, first(crange))
    eliminar_valrep_ordenar!(xx1,flimit,ϵ)
    
    j=0
    for ic in 2:length(crange)
        c = crange[ic]
        ciclosestables!(xx2, f, nit, nout, c)
        eliminar_valrep_ordenar!(xx2,flimit,ϵ)
        
        if length(xx1)!= length(xx2)
            push!(rc, c)
            push!(rf,(first(xx1)+first(xx2))/2)
            j=j+1
            if j==2
                break
            end
        end
        xx1 = xx2
        xx2 = Vector{Float64}(nout)
    
    end
    
    return  rc,rf
end

primera_bifurcacion (generic function with 2 methods)

Ya estamos en condición de trabajar la función final para encontrar las bifurcaciones.

In [11]:

function encontrar_bifurcaciones(f,crange,flimit,N)
    
    #Vectores vacíos para almacenar los resultados.
    Rc=Float64[]
    Rf=Float64[]
    
    ϵ   = 1e-05 #margen de error
    nit  = 2000
    
    #Primer valor:
    f0 = Vector{Float64}(256)
    
    ciclosestables!(f0, f, nit, 2, first(crange))
    eliminar_valrep_ordenar!(f0)
    f0=f0[1]
    
    #Primera bifurcación:
    c12,f12=primera_bifurcacion(f, nit, 4, crange)
    push!(Rc,c12[1])
    push!(Rf,f12[1])
    
    #Factores para encontrar los nuevos crange y flimit.
    
    Δc = abs(c12[1]-c12[2])
    Δf = abs(f12[1]-f12[2])
    
    αc = abs(Δc/(first(crange)-c12[1]))
    αf = abs(Δf/(f0-f12[1]))
    
    crangen = c12[1]-0.1*Δc : -Δc/1000 : c12[1]-((1+αc)*Δc)
    flimitn = [f12[1]-(1+αf)*Δf , f12[1]]
    
    for i in 2:N
        
        #Aquí se encuentran las bifurcaciones dentro del los nuevos intervalos.
        
        c12,f12 = primera_bifurcacion(f, nit, 2^(i+3), crangen, flimitn, ϵ)
        push!(Rc,c12[1])
        push!(Rf,f12[1])
        
        #Redefinición de los intervalos.
        Δc=αc*Δc
        Δf=αf*Δf
        
        crangen = c12[1]-0.1*Δc : -Δc/1000 : c12[1]-((1+αc)*Δc)
        flimitn = [f12[1]-(1+αf)*Δf , f12[1]]
        
    end
    #Resultados.
    Rc,Rf    
    

end
    

encontrar_bifurcaciones (generic function with 1 method)

Trabajamos con nuestra función:

In [12]:
Qc(x,c) = x^2 + c

flimit = [-2.0,2.0]
crange  = 0.25:-1/2^10:-2.0

Rc,Rf=encontrar_bifurcaciones(Qc,crange,flimit,10);

Verificamos, los datos mediante una gráfica.

In [13]:
figure(figsize=(6,4))
plot(cc, ff, "b,")

plot(c0,f0,"go",label=L"f_0")

plot(Rc,Rf,"r+",)




plot()
xlabel(L"c")
ylabel(L"x_\infty")
legend()
title("Fig. 3")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[13], in expression starting on line 1

Las bifurcaciones encontradas corresponden a:

In [14]:
Rc

10-element Array{Float64,1}:
 -0.74707
 -1.24852
 -1.36754
 -1.39388
 -1.40086
 -1.40411
 -1.40575
 -1.40658
 -1.40699
 -1.4072 

Ahora sí, hacemos el ejercicio con $f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}}$:

In [15]:
[(Rc[i]-Rc[i+1])/(Rc[i+1]-Rc[i+2]) for i in 1:length(Rc)-2]

8-element Array{Any,1}:
 4.21315
 4.51974
 3.7723 
 2.14372
 1.98638
 1.98638
 1.98638
 1.98638

Como se puede apreciar, converge a 1.98638

**Ejercicio 2:**

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. ¿Cómo se comparan los valores obtenidos de $f_n$?

Primero hacemos:

$$S_c(x) = x$$

$$ \sin(x) = \frac{x}{c} $$

Obsevamos que se tienen muchas raíces que dependen del valor de c, sólo en el caso  $c \leq \pm 1$ se tiene una sola raíz, $x=0$, sin importar el valor de $c$.

Si derivamos el mapeo:

$$Sc'=c\cos(x)$$

Para $|Sc'|\leq 1$, es decir, atractivo, requerimos que:

$$-1\leq c \leq 1$$

Entonces:

$$-1 \leq c \cos(x) \leq 1 $$

$$-\frac{1}{c} \leq  \cos(x) \leq \frac{1}{c} $$

$$-\arccos \left ( \frac{1}{c} \right ) \leq  x \leq \arccos \left ( \frac{1}{c} \right ) $$



La función `ciclos_estables` está centrada en un valor inicial $x_0=0.0$, para no modificar la sintaxis, usamos $S_c(x) = c \cos(x)$, sabiendo que $\sin(x)=\cos(\pi/2-x)$

In [16]:
Sc(x,c) = c*cos(x)

crange = -1:-1/2^10:-3
ff = diagbifurc(Sc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';

In [17]:
figure(figsize=(6,4))
plot(cc, ff, "b,")

plot()
xlabel(L"c")
ylabel(L"x_\infty")
legend()
title("Fig. 4")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[17], in expression starting on line 1

In [18]:
Sc(x,c) = c*cos(x)

crange = -1:-1/2^10:-3
ff = diagbifurc(Sc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';
flimi=[-3.0,3.0]


Rc,Rf=encontrar_bifurcaciones(Sc,crange,flimit,6);

In [19]:
[(Rc[i]-Rc[i+1])/(Rc[i+1]-Rc[i+2]) for i in 1:length(Rc)-2]

4-element Array{Any,1}:
 4.33181 
 0.878798
 0.62069 
 0.62069 

Observamos un diagrama muy similar, con un convergencia en 0.62069.

**Ejercicio 3:**

Como se ve en la Fig. 1, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que $x=0$ esté en el ciclo de periodo $2^n$ marca los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

¿A qué converge la secuencia $f_n$, definida ahora con los valores $C_n$.

De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es menor; a ese punto lo identificamos como $d_n$. Calcular numéricamente a dónde converge la secuencia $d_n/d_{n+1}$.

Requerimos una función distinta.

In [26]:

"""
    primera_intersección(f, nit, nout, crange,ϵ)

Dando una función de la forma `f(x,c)`, después de
iterar el mapeo `nit+nout` veces, evaluando sólo las 
último valores `nout`, devuelve la primera `c`, con
valor |c|<=ϵ en `crange` así, como el 2° falor de f
mas cercano a 0 con esa `c`.

"""

function primera_intersección(f, nit, nout, crange,ϵ)
    
    #Variables para almacenar el resultado.
    rc    = Float64
    rf    = Float64
    
    
    for ic in 1:1:length(crange)
        c = crange[ic]
        
        xx1 = Vector{Float64}(nout)
        
        #Aquí calculamos el valor absoluto...
        ciclosestables!(xx1, f, nit, nout, c)
        xx1=abs(xx1)
        eliminar_valrep_ordenar!(xx1)  #Ordenamos y quitamos valores repetidos.
        
        if xx1[1] <= ϵ  #Evaluamos sólo el primer valor, puesto que el vector está ordenado.
            rc = c
            if length(xx1)==1
                rf=xx1[1]
                break
            else
                rf = xx1[2]  #Almacenamos el segundo valor, para la última parte del ejercicio.
                break
            end
        end
    end
    
    rc,rf

end

primera_intersección (generic function with 1 method)

In [21]:
primera_intersección(Qc, 2000, 264, crange, 1e-3)

(-1.0,1.0)

In [22]:

"""
    encontrar_intersecciones(f,crange,N)

Dando una función de la forma `f(x,c)`, encuentra las
primeras `N ` c's en el rango `crange` dado que `f`=0

También devuelve las f's más cercanas a 0.0.

"""

function encontrar_intersecciones(f,crange,N)
    
    #Vectores vacíos para almacenar los resultados.
    Rc=Float64[]
    Rf=Float64[]
    
    ϵ   = 1e-04 #margen de error
    nit  = 2000
    
    crangen = crange

    for i in 1:N
        
        #Aquí se encuentran las bifurcaciones dentro del los nuevos intervalos.
        
        rc,rf = primera_intersección(f, nit, 2^(i+3), crangen,ϵ)
        push!(Rc,rc)
        push!(Rf,rf)
        
        #Redefinición de los intervalos.
        Δc=abs(rc-last(crange))
        
        crangen = rc-0.05*Δc : -Δc/3000 : last(crange)
        
    end
    #Resultados.
    Rc,Rf    
    

end

encontrar_intersecciones (generic function with 1 method)

In [23]:
Qc(x,c) = x^2 + c

flimit = [-2.0,2.0]
crange  = 0.25:-1/2^10:-2.0

Rc,Rf=encontrar_intersecciones(Qc,crange,14);

Calculamos:

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

In [24]:
[(Rc[i]-Rc[i+1])/(Rc[i+1]-Rc[i+2]) for i in 1:length(Rc)-2]

12-element Array{Any,1}:
 3.21888 
 3.08683 
 2.93075 
 0.720313
 1.88184 
 1.0057  
 1.10447 
 1.05263 
 1.04566 
 0.908589
 1.23073 
 1.01276 

Díficilmente se obseva una convergencia, como era de esperarse en el dibujo.

Y ahora: 
$d_n/d_{n+1}$

In [25]:
[Rc[i]/(Rc[i+1]) for i in 1:length(Rc)-1]

13-element Array{Any,1}:
 -0.0     
  0.76297 
  0.928688
  0.976246
  0.968075
  0.983318
  0.983683
  0.985442
  0.986358
  0.987122
  0.986025
  0.988772
  0.989035

No se observa una convergencia, pero parece que el valor es asintótico a 1.0